In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from code_source.dataframe_prep import get_cols_names
from code_source import data_prep
from code_source import metrics
from code_source import learning

# dataframe preparation

In [2]:
data=pd.read_csv('data/Dataset/Dataset/Training/Features_Variant_1.csv', header=None).drop_duplicates()
data=get_cols_names(data)
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

# data preparation

In [3]:
#'page_cat' to dummy because it is a category
#data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
#data=data.drop(columns=['page_cat'])

In [4]:
#data shuffling
data=shuffle(data).reset_index(drop=True)

# learning

In [5]:
#set algo params:
lam=1e-2
gamma=0 #now we needn't it
terms_num=10000
max_iter=1

# Cross-val

In [6]:
data.iloc[:,:-1], mean, std =data_prep.normalize(data.iloc[:,:-1])

res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])

    #features normalization (xi-mean)/std_err:
    #train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    #test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    
    params=learning.gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=learning.get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=metrics.custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=metrics.custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

Iterations number is:1
The maximum number of iterations was reached.
RMSE is:36.16586621175759
R2 is:-0.04248700494167701
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:36.610781354680654
R2 is:-0.0397463838277583
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:36.858080117071715
R2 is:-0.0374917566999422
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:35.77780990136088
R2 is:-0.04052936625895254
Iterations number is:1
The maximum number of iterations was reached.
RMSE is:35.619666581539576
R2 is:-0.04258776838937983


In [7]:
text_idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
text_idxs=np.append(text_idxs,'w0')
text_idxs=np.insert(text_idxs, 0, ['RMSE', 'r2'])
res.index=text_idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], axis=1)

In [8]:
res

,1,2,3,4,5,mean,std
RMS,36.500242,34.567896,33.334860,37.909203,38.472841,36.157008,2.181792
r2,-0.039481,-0.044624,-0.045713,-0.041881,-0.033260,-0.040992,0.004958
w1,0.000541,-0.002531,0.012077,-0.003402,-0.002513,0.000835,0.006461
w2,-0.000571,0.002916,0.002866,-0.005062,-0.005520,-0.001074,0.004104
w3,-0.000947,-0.000642,0.008581,-0.002116,-0.001799,0.000615,0.004493
w4,0.000520,-0.004795,0.001065,0.001323,-0.003284,-0.001034,0.002810
w5,0.001072,0.005218,0.001614,-0.004067,-0.002673,0.000233,0.003688
w6,-0.003684,0.002436,0.005733,-0.001945,0.002842,0.001076,0.003822
w7,-0.000598,0.002015,0.005930,0.001899,0.001691,0.002187,0.002352
w8,0.000186,0.001728,0.005821,0.004354,0.000326,0.002483,0.002507
